![MBITSchool](https://i.imgur.com/UiDMkO3.png)

### Proyecto de Consolidación APIs y Web Scraping

##### Alejandro Paredero - paredero@mbitschool.com

**El proyecto de consolidación de Web Scraping, API REST y Streamlit pretende que de forma autónoma combinemos proyectos de captura, procesamiento de datos y visualización basados en un sistema o entre dos o más distintos.**

Aquí la creatividad es muy importante ✔️. Una vez resuelto los retos propuestos siéntete libre de extenderlos con características adicionales.

⚠️ **ATENCIÓN**: <u>Ten a mano las presentaciones y los cuadernos resueltos de las sesiones anteriores, te serán de gran ayuda.</u>

**Los ejercicios EXTRA son opcionales**. Si dais vuestro consentimiento tras la fecha de cierre, podré hacer publicaciones con capturas de pantalla en redes como LinkedIn con objeto de promocionaros.

Para dudas tenéis un foro en el campus o mi correo electrónico 📧 paredero@mbitschool.com

**Datos del alumno:**

- **Nombre:** `Daniel Herraiz Tello`
- **Consentimiento sobre ejercicios EXTRA:** Si realizas el ejercicio, ¿permites que pueda publicar el contenido en foto/vídeo si el resultado es relevante? `SI`
- **Comentario:** Tras realizar los diferentes retos, ¿qué te han parecido? ¿qué problemas has encontrado?

```
Escribe tu comentario aquí
```

### Reto 1 - BeautifulSoup y Streamlit: Capturar estadísticas de equipos de hockey

Vista la web https://www.scrapethissite.com/pages/forms/. Examina como se comporta la URL con la paginación.  El objetivo es capturar la información de las primeras 10 páginas y crear una aplicación sencilla en Streamlit que permita mostrar la información de las columnas "Team name", "Year", "Win", "Loses" y "+/-"

##### Ejercicio:
* Captura el los términos listados anteriormente:  "Team name", "Year", "Win", "Loses" y "+/-" , de al menos las primeras 8 páginas.
* Almacena el resultado en un **dataframe** de Steramlit.
* Implementa el cacheo de contenido de las URL (Revisa la presentación) para evitar consultas repetidas a las URL.
* **Extra**: Permite que el usuario elija el rango de página mínima y máxima a capturar contenido
* **Extra**: Permite que el usuario elija filtrar por equipos cuyo "+/-" sea superior a una cifra determinada.

*PISTA: La página 1 tiene la estructura `https://www.scrapethissite.com/pages/forms/?page_num=1` , la página 2 `https://www.scrapethissite.com/pages/forms/?page_num=2` y así sucesivamente.*


##### Comentarios
* Si realizo una búsqueda incorrecta se queda pensando mucho tiempo.
* La columna +/- requiere especial atención y me ha dado algunos problemas al variar el nombre de la clase según >0 o <0
* Al obtener el rango de páginas, a veces se queda bloqueado y tengo que reiniciar el kernel, sin un patrón claro. Añadiendo retraso entre llamadas tampoco se arregla

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import selenium
import re
import time

In [2]:
#Creo un diccionario con el nº de la página como clave y el contenido como valor
cachedPageContent = {}
#función para obtener el contenido de la página de cache o via request
def getPageContent (pageNumber):
    if pageNumber in cachedPageContent:
        print(f"pag {pageNumber} obtenida desde cache")
        return cachedPageContent[pageNumber]
    else:  
        time.sleep(0.2)
        url = f"https://www.scrapethissite.com/pages/forms/?page_num={pageNumber}"
        page = requests.get(url)
        if page.status_code != 200:
            raise Exception(f"Expected 200, got {page.status_code}")
        soup = BeautifulSoup(page.content, "html.parser")
        cachedPageContent[pageNumber] = soup
        return soup  

In [3]:
#función para obtener el dataframe de una página
def getDfFromPage (pageNumber, minGoalDif):
    table = getPageContent(pageNumber).find("table", class_="table")
    #Puedo extraer todas las columnas
    columnNames = [th.get_text(strip=True) for th in table.find_all("th")]
    #O forzar las columnas sugeridas 
    columnNames = ["Team name","Year","Wins","Losses","+/-"]
    #Extraigo las filas
    rows = table.find_all("tr", class_="team")
    #Extraigo la información de cada campo en cada fila
    teams = []
    for row in rows:
        #Extra; primero aplico el filtro +/-
        if int(row.find("td", class_=re.compile("diff")).get_text(strip=True)) > minGoalDif:
            #filtro por tag y por clase teniendo en cuenta las columnas solicitadas
            values = [td.get_text(strip=True) for td in row.find_all("td", class_=["name","year","wins","losses",re.compile("diff")])]
            teams.append(values)  
    return pd.DataFrame(teams, columns=columnNames) 
        

In [4]:
testteams = []
htmltest = """                       <td class="diff text-success">
                            -35
                        </td>"""
soup = BeautifulSoup(htmltest, 'html.parser')
all_text = soup.get_text(strip=True)
print(all_text)
valuestest = [td.get_text(strip=True) for td in soup.find_all("td", class_=["name","year","wins","losses","diff text-success"])]
testteams.append(valuestest)  
print(testteams)
testdf = pd.DataFrame(testteams)
testdf.head()

-35
[['-35']]


,0
0,-35


In [5]:
#Función para obtener un dataframe obtenido de un rango de páginas
def getDfFromPageRange (firstPage, lastPage, minGoalDif):
    resultDf = pd.DataFrame()
    #try:
    for i in range(firstPage, lastPage + 1):
        pageDf = getDfFromPage(i,minGoalDif)
        if resultDf.empty:
            resultDf = pageDf.copy()
        else:
            resultDf = pd.concat([resultDf, pageDf], ignore_index = True)
    #except Exception:
        #print ("Rango inválido de páginas, prueba otros valores")
    return resultDf

In [8]:
df_pag1 = getDfFromPage(1, 15)
print(df_pag1.shape)
df_pag1.head(10)

(9, 5)


,Team name,Year,Wins,Losses,+/-
0,Boston Bruins,1990,44,24,35
1,Calgary Flames,1990,46,26,81
2,Chicago Blackhawks,1990,49,23,73
3,Los Angeles Kings,1990,46,24,86
4,Montreal Canadiens,1990,39,30,24
5,New York Rangers,1990,36,31,32
6,Pittsburgh Penguins,1990,41,33,37
7,St. Louis Blues,1990,47,22,60
8,Chicago Blackhawks,1991,36,29,21


In [ ]:
df_pag1_8 = getDfFromPageRange(1,8,15)
print(df_pag1_8.shape)
df_pag1_8.head()

pag 1 obtenida desde cache
pag 2 obtenida desde cache
pag 3 obtenida desde cache
pag 4 obtenida desde cache
pag 5 obtenida desde cache
pag 6 obtenida desde cache
pag 7 obtenida desde cache
pag 8 obtenida desde cache
(77, 5)


,Team name,Year,Wins,Losses,+/-
0,Boston Bruins,1990,44,24,35
1,Calgary Flames,1990,46,26,81
2,Chicago Blackhawks,1990,49,23,73
3,Los Angeles Kings,1990,46,24,86
4,Montreal Canadiens,1990,39,30,24


: 

### Reto 2 - Selenium: Captura del valor del oro

Como vimos en clase, hay páginas que implementan medidas para evitar el Web Scraping. En la web de https://www.inversoro.es/precio-del-oro/precio-oro-hoy/ apreciamos cómo la página web cara un valor antiguo y pasados unos segundos actualiza. 

Esto hace que bibliotecas como `request` + `BeautifulSoup` no nos sirva aquí ya que capturamos el instante t=0. Sin embargo, con *Selenium* podemos interactuar en tiempo real mientras el navegador se ejecuta.

#### Ejercicio
Inspecciona la web, obteniendo la etiqueta HTML donde el valor aparece representado y crea un script que utilice Selenium, cargando la página y capturando  dicho valor en tiempo real cada 5 segundos en un periodo de 1 minuto. Almacena el resultado de dicho valor y del timestamp correspondiente (*[ayuda](https://www.geeksforgeeks.org/get-current-timestamp-using-python/)*)

In [ ]:
import selenium

### Reto EXTRA - Selenium, BeautifulSoup y Streamlit: El comparador (simple) de precios 📊📉

Los comparadores de precios <u>es uno de los nichos de mercado más lucrativos en Internet.</u> Dado un producto o servicio a nivel de usuario nos beneficiamos del precio más bajo existente y a nivel de empresa éstas obtienen grandes beneficios gracias a sistemas de referidos o de comisiones por venta realizada.

En esta ocasión vamos a realizar un simple comparador de precios basado en el número ISBN. Un ISBN es un código normalizado internacional para libros (International Standard Book Number). Estos estaban compuestos por 10 dígitos hasta diciembre de 2006 pero, desde enero de 2007, tienen una extensión de 13 dígitos. 


Por ejemplo, el ISBN [9788478884452](https://www.google.es/search?q=9788478884452) (haz click) corresponde al libro "Harry Potter y la piedra filosofal".

Dado que no disponemos de acceso a APIs de las tiendas principales <u>capturaremos el precio a través de técnicas de Web Sraping.</u>

##### Ejercicio 

El objetivo es realizar la consulta de un ISBN, por ejemplo el de "Harry Potter y la Piedra filosofal" en al menos **tres** de las siguientes tiendas propuestas. Captura el primer resultado en EUROS, y devuelve cómo resultado cuál tiene el precio más bajo. 

Crea un sencillo interfaz en Streamlit que pregunte por el ISBN de un número (entre 10 y 13 dígitos) y devuelva como resultado el primer elemento de cada tienda examinada, con el título, la imagen (si la hubiese), el precio y un enlace para hacer click.

Como puntos extra:
* Incluye en la comparativa más de las 3 tiendas propuestas.
* Incluye por texto posteriormente cuál es la diferencia de precio en euros y en porcentaje respecto al valor más bajo detectado para saber cuánto nos estamos ahorrando.
* Crea un diagrama de barras con el precio en cada tienda para representar visualmente el ahorro en precio.

Ejemplo de tiendas propuestas:
- https://www.casadellibro.com/
- https://www.libreriacentral.com/
- https://www.iberlibro.com/
- https://www.amazon.es/
- https://ebay.es
- https://www.elcorteingles.es/

Recuerda que para cada página debes realizar ingeniería inversa, averiguando cómo se comportan las URLs de cada sitio web para hacer una búsqueda directa.

**Pista**: Utiliza Selenium para capturar datos si te resulta request/BeautifulSoup complicado de utilizar.

**Extra**: *¿Conoces alguna otra página donde comprar libros? Inclúyela en el comprador*

**Extra 2**: *¿Cómo podríamos obtener una evolución del precio durante una semana?*

In [ ]:
import json
import requests
import streamlit as st
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep

In [ ]:
## ENTREGA AQUÍ EL CÓDIGO PYTHON DE LA APLICACIÓN STREAMLIT